# Part 1: How to get the same New York 2017 dataset with 157269 rows that is usually used in articles

All original datasets can be downloaded from here: https://www.consumerfinance.gov/data-research/hmda/historic-data/ (use 'All Records' and 'Plain language' options)

Download datasets Data_lending and hmda_2017_ny_all-records_labels for this guide from here: https://drive.google.com/drive/folders/1ctZxncnxScM5q49V3X2iaFjmW6K9CVf7?usp=drive_link

In [1]:
import pandas as pd
import numpy as np

data_article = pd.read_csv("Data_lending.csv", low_memory=False)
print("New York Article dataset has {} rows and {} columns".format(data_article.shape[0], data_article.shape[1]))

data_original = pd.read_csv("hmda_2017_ny_all-records_labels.csv", low_memory=False)
print("New York Original dataset has {} rows and {} columns".format(data_original.shape[0], data_original.shape[1]))

New York Article dataset has 157269 rows and 60 columns
New York Original dataset has 446902 rows and 78 columns


We begin by identifying and removing any columns that are unique to the original New York dataset, so they are not used in New York Article data. 
Also we remove 3 artificial columns (response variable which has the same meaning as action_taken and 2 columns with numbers of rows) from the New York Article dataset.

In [2]:
columns_article = set(data_article.columns)
columns_original = set(data_original.columns)

unique_article = columns_article - columns_original
unique_original = columns_original - columns_article

print("Columns presented only in New York Article dataset:", unique_article)
print("Columns presented only in New York Original dataset:", unique_original)

data_article = data_article.drop(columns=unique_article)
data_original = data_original.drop(columns=unique_original)

Columns presented only in New York Article dataset: {'response', 'X', 'Unnamed: 0'}
Columns presented only in New York Original dataset: {'preapproval', 'state_abbr', 'edit_status', 'owner_occupancy_name', 'preapproval_name', 'hoepa_status_name', 'hoepa_status', 'edit_status_name', 'state_name', 'sequence_number', 'agency_abbr', 'owner_occupancy', 'state_code', 'agency_code', 'as_of_year', 'applicant_sex', 'co_applicant_sex', 'property_type_name', 'property_type', 'application_date_indicator', 'agency_name'}


Next we identify which categorical values were excluded. We apply the same method to each object-type column (excluding respondent_id). And after we remove rows with these values from original New York data.

In [3]:
object_columns = data_article.select_dtypes(include='object').columns
object_columns = [col for col in object_columns if col != 'respondent_id']

for col in object_columns:
    article = set(data_article[col].dropna().unique())
    original = set(data_original[col].dropna().unique())
    only_original = original - article

    if only_original:
        print(f"\nColumn: {col}")
        print("Only in New York Original:", only_original)
        
        data_original = data_original[~data_original[col].isin(only_original)]


Column: loan_type_name
Only in New York Original: {'FSA/RHS-guaranteed', 'VA-guaranteed'}

Column: loan_purpose_name
Only in New York Original: {'Home improvement'}

Column: action_taken_name
Only in New York Original: {'File closed for incompleteness', 'Preapproval request denied by financial institution', 'Loan purchased by the institution', 'Application withdrawn by applicant', 'Application denied by financial institution', 'Preapproval request approved but not accepted'}

Column: applicant_race_name_1
Only in New York Original: {'American Indian or Alaska Native', 'Native Hawaiian or Other Pacific Islander', 'Information not provided by applicant in mail, Internet, or telephone application', 'Not applicable', 'Asian'}

Column: applicant_sex_name
Only in New York Original: {'Information not provided by applicant in mail, Internet, or telephone application', 'Not applicable'}


In [4]:
print(data_article.shape)
print(data_original.shape)

(157269, 57)
(162282, 57)


We are left with only 5,013 differing rows between the two datasets. Now we will compare each column and determine where the amount of unique values differs by the exact value of rows.

In [5]:
expected_diff = data_original.shape[0] - data_article.shape[0]

for col in data_original.columns:
    values_in_article = set(data_article[col].dropna().unique())
    mask = ~data_original[col].isin(values_in_article)
    count = mask.sum()

    if count == expected_diff:
        print(f"Column: {col} — {count} values not found in New York Article (the exact amount)")

article = set(data_article['applicant_income_000s'])
original = set(data_original['applicant_income_000s'])

only_original = original - article
unique_original = data_original[data_original['applicant_income_000s'].isin(only_original)]
print(unique_original['applicant_income_000s'].value_counts())

Column: applicant_income_000s — 5013 values not found in New York Article (the exact amount)
Series([], Name: count, dtype: int64)


We found the column which is responsible for the remaining 5,013 rows. As the final cleaning step, we remove these rows from the dataset which results in two fully matched datasets (all these rows are missing data).

In [6]:
data_original['applicant_income_000s'] = data_original['applicant_income_000s'].replace('', np.nan)
data_original = data_original[~data_original['applicant_income_000s'].isna()]

print(data_article.shape)
print(data_original.shape)

(157269, 57)
(157269, 57)


Overall, there are some differences in numerical values within certain categorical columns and in client identifiers, but both datasets are fundamentally equivalent.

## Part 2: Fixing action_taken Variable


In [7]:
data_original = pd.read_csv("hmda_2017_ny_all-records_labels.csv", low_memory=False)
print("New York Original dataset has {} rows and {} columns".format(data_original.shape[0], data_original.shape[1]))

data_article = pd.read_csv("Data_lending.csv", low_memory=False)

print(f'Response definition in New York Article {data_article['action_taken_name'].value_counts()}')
print(f'Response definition in New York Original {data_original['action_taken_name'].value_counts()}')

New York Original dataset has 446902 rows and 78 columns
Response definition in New York Article action_taken_name
Loan originated                          147255
Application approved but not accepted     10014
Name: count, dtype: int64
Response definition in New York Original action_taken_name
Loan originated                                        236499
Application denied by financial institution             68255
Loan purchased by the institution                       59584
Application withdrawn by applicant                      47506
File closed for incompleteness                          19397
Application approved but not accepted                   15647
Preapproval request denied by financial institution        11
Preapproval request approved but not accepted               3
Name: count, dtype: int64


When reviewing the credit decision data in the New York Article dataset, we can see that cases labeled as "Application approved but not accepted" were used instead of "Application denied by financial institution."

According to the official HMDA guidance https://www.consumerfinance.gov/rules-policy/regulations/1003/interp-4/, the category "Application approved but not accepted" may include situations where the applicant chooses not to proceed by his own decision or other non-standard scenarios. So it is more appropriate to use "Application denied by financial institution" as a more direct indicator of credit denial.

So to replicate the dataset structure used in Part 1, but with a corrected action_taken variable, we only need to modify a small part of code.

In [8]:
# First step
columns_article = set(data_article.columns)
columns_original = set(data_original.columns)

unique_article = columns_article - columns_original
unique_original = columns_original - columns_article

print("Columns presented only in New York Article dataset:", unique_article)
print("Columns presented only in New York Original dataset:", unique_original)

data_article = data_article.drop(columns=unique_article)
data_original = data_original.drop(columns=unique_original)

# Second step
object_columns = data_article.select_dtypes(include='object').columns
object_columns = [col for col in object_columns if col != 'respondent_id']

for col in object_columns:
    article = set(data_article[col].dropna().unique())
    original = set(data_original[col].dropna().unique())
    only_original = original - article

    if only_original:
        print(f"\nColumn: {col}")
        print("Only in New York Original:", only_original)

        # Here is the additional code for fixing response variable
        if col == 'action_taken_name':
            values_to_delete = {'Application approved but not accepted', 'Loan purchased by the institution',
                                'File closed for incompleteness', 'Preapproval request approved but not accepted',
                                'Preapproval request denied by financial institution', 'Application withdrawn by applicant'}
            data_original = data_original[~data_original[col].isin(values_to_delete)]
        else:
            # Default behavior for other columns
            data_original = data_original[~data_original[col].isin(only_original)]
# Third step
data_original['applicant_income_000s'] = data_original['applicant_income_000s'].replace('', np.nan)
data_original = data_original[~data_original['applicant_income_000s'].isna()]

print(data_article.shape)
print(data_original.shape)

Columns presented only in New York Article dataset: {'response', 'X', 'Unnamed: 0'}
Columns presented only in New York Original dataset: {'preapproval', 'state_abbr', 'edit_status', 'owner_occupancy_name', 'preapproval_name', 'hoepa_status_name', 'hoepa_status', 'edit_status_name', 'state_name', 'sequence_number', 'agency_abbr', 'owner_occupancy', 'state_code', 'agency_code', 'as_of_year', 'applicant_sex', 'co_applicant_sex', 'property_type_name', 'property_type', 'application_date_indicator', 'agency_name'}

Column: loan_type_name
Only in New York Original: {'FSA/RHS-guaranteed', 'VA-guaranteed'}

Column: loan_purpose_name
Only in New York Original: {'Home improvement'}

Column: action_taken_name
Only in New York Original: {'File closed for incompleteness', 'Preapproval request denied by financial institution', 'Loan purchased by the institution', 'Application withdrawn by applicant', 'Application denied by financial institution', 'Preapproval request approved but not accepted'}

Colu

### Part 3: My Custom HMDA Data Processing Procedure

This code is compatible with any HMDA dataset downloaded from the official website, including data up to the year 2017.

You can customize code by selecting which columns and values to filter out—simply adjust the values in the values_to_remove and columns_to_remove dictionaries within the code (but in some cases you will need to change missing values cleaning logic due to some almost empty columns).

If you’d like to follow the approach used in the New York dataset from the articles, remove values based on Part 2 output.

For a detailed explanation of the numeric codes (the only changed variable is action_taken: 1->0 and 3->1) used in HMDA data check this link: https://files.consumerfinance.gov/hmda-historic-data-dictionaries/lar_record_codes.pdf

In [9]:
data_lending = pd.read_csv("hmda_2017_ny_all-records_labels.csv", low_memory=False)
print(f'Original shape: {data_lending.shape}')

# Values to remove
values_to_remove = {
    'action_taken': [2, 4, 5, 6, 7, 8],
    'loan_type': [3, 4],
    'applicant_race_1': [1, 2, 4, 6, 7],
    'lien_status': [3, 4],
    'applicant_sex': [3, 4],
    'co_applicant_sex': [3, 4],
    'co_applicant_race_1': [1, 2, 4, 6, 7],
    'applicant_ethnicity': [3, 4],
    'co_applicant_ethnicity': [3, 4]
}

# Filter
data_lending_short = data_lending.copy()
for col, values in values_to_remove.items():
    data_lending_short = data_lending_short[~data_lending_short[col].isin(values)]

# Delete columns
columns_to_remove = ['as_of_year', 'agency_name', 'agency_abbr', 'agency_code', 'property_type_name',
       'property_type', 'owner_occupancy_name', 'owner_occupancy', 'preapproval_name', 'preapproval',
       'state_name', 'state_abbr', 'state_code', 'applicant_race_name_2', 'applicant_race_2',
       'applicant_race_name_3', 'applicant_race_3', 'applicant_race_name_4', 'applicant_race_4',
       'applicant_race_name_5', 'applicant_race_5', 'co_applicant_race_name_2', 'co_applicant_race_2',
       'co_applicant_race_name_3', 'co_applicant_race_3', 'co_applicant_race_name_4', 'co_applicant_race_4',
       'co_applicant_race_name_5', 'co_applicant_race_5', 'purchaser_type_name', 'purchaser_type',
       'denial_reason_name_1', 'denial_reason_1', 'denial_reason_name_2', 'denial_reason_2', 'denial_reason_name_3',
       'denial_reason_3', 'rate_spread', 'hoepa_status_name', 'hoepa_status','edit_status_name', 'edit_status',
       'sequence_number', 'application_date_indicator', 'respondent_id', 'loan_type_name', 'loan_purpose_name',
       'action_taken_name', 'msamd_name', 'county_name', 'applicant_ethnicity_name', 'co_applicant_ethnicity_name',
       'applicant_race_name_1', 'co_applicant_race_name_1', 'applicant_sex_name', 'co_applicant_sex_name',
       'lien_status_name']

data_lending_dropped = data_lending_short.drop(columns=columns_to_remove)

# Clean and refactor data
data_lending_clean = data_lending_dropped.dropna(axis='index')
data_lending_clean.loc[:, 'action_taken'] = data_lending_clean['action_taken'].map({1: 0, 3: 1})
print(f'Final shape after cleaning: {data_lending_clean.shape}')

# Save
data_lending_clean.to_csv("cleaned_data.csv", index=False)

Original shape: (446902, 78)
Final shape after cleaning: (171477, 21)
